
Quando l’utente fornisce una descrizione del tipo “Mi serve un lungo forte a rimbalzo e stoppate”, il sistema analizza la frase per individuarne le parole chiave e interpretarli come concetti tecnici. I termini rilevanti vengono quindi mappati automaticamente sulle feature numeriche corrispondenti: ad esempio “tiratore da tre” è collegato alla statistica 3P%, “liberi” al FT%, “rimbalzista” ai rimbalzi offensivi e difensivi, “playmaker” agli assist, “difensore” alle metriche difensive come BLK, STL o Defensive WS. In questa fase vengono anche assegnati pesi ai vari concetti, così da modellare l’importanza relativa delle diverse caratteristiche.


In [1]:
import os
import re
import numpy as np
import pandas as pd
import joblib

from sklearn.metrics.pairwise import cosine_similarity

# ============================
# 1. Caricamento indice esistente
# ============================

MODELS_DIR = "../data/models"
VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH = os.path.join(MODELS_DIR, "index_metadata.csv")

vectorizer = joblib.load(VECTORIZER_PATH)
tfidf_matrix = joblib.load(MATRIX_PATH)
metadata_df = pd.read_csv(META_PATH)

print("Indice caricato.")
print("TF-IDF matrix shape:", tfidf_matrix.shape)
print("Metadata shape:", metadata_df.shape)

# ============================
# 2. Normalizzazione
# ============================

def normalize_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text


# ============================
# 3. Regole di parsing semantico
# ============================

# Skill principali: tiri, rimbalzi, difesa, playmaking
SKILL_PATTERNS = {
    "shooting_3": [
        "tiratore da 3", "ottime percentuali da 3", "buon tiratore da 3",
        "da 3 punti", "tripla", "3 punti"
    ],
    "ft": [
        "buon ft", "ottimo ft", "buon tiratore di liberi", "ottimo tiratore di liberi",
        "buon ft%", "buon ft", "tiri liberi", "liberi"
    ],
    "reb": [
        "rimbalzista", "forte rimbalzista", "buon rimbalzista", "rimbalzi",
        "dominante a rimbalzo", "forte a rimbalzo"
    ],
    "playmaking": [
        "playmaker", "buon passatore", "assist", "creatore di gioco",
        "regista", "fa girare la squadra"
    ],
    "scorer": [
        "realizzatore", "scorer", "tanti punti", "tanti punti a partita",
        "prima opzione offensiva"
    ]
}

# Espansione di testo che useremo per ogni concetto (parole che esistono nei profili)
SEMANTIC_EXPANSIONS = {
    "shooting_3": "ottimo tiratore da 3 3 punti 3P% buon tiratore da 3",
    "ft": "tiri liberi FT% buon tiratore ai liberi eccellente tiratore ai liberi",
    "reb": "rimbalzi forte rimbalzista buon rimbalzista",
    "playmaking": "assist buon passatore ottimo playmaker",
    "scorer": "segna molti punti realizzatore punti a partita"
}

# Peso (quante volte ripetere l’espansione nel testo pesato)
SEMANTIC_WEIGHTS = {
    "shooting_3": 4,
    "ft": 3,
    "reb": 3,
    "playmaking": 3,
    "scorer": 3
}

#Status del giocatore, se è in attività o meno
STATUS_PATTERNS = {
    "Active": ["in attività", "ancora in attività", "ancora gioca", "sta ancora giocando"],
    "Retired": ["ritirato", "non gioca più", "ha smesso", "in pensione"]
}

# ============================
# 4. Parsing della query -> intenti
# ============================

def parse_query_intents(raw_query):
    """
    Dato l'input naturale dell'utente, rileva:
    - set di skill richieste (shooting_3, reb_off, difesa, playmaking, ...)
    Restituisce un dict con:
      {
        "normalized_query": ...,
        "skills": [...],
      }
    """
    q = normalize_text(raw_query)
    detected_skills = set()
    detected_status = None

    # Stato (attivo / ritirato)
    # Stato
    for status, patterns in STATUS_PATTERNS.items():
        for p in patterns:
            if p in q:
                detected_status = status

        
      # Skill
    for skill, patterns in SKILL_PATTERNS.items():
        for p in patterns:
            if p in q:
                detected_skills.add(skill)
                break

    return {
        "normalized_query": q,
        "skills": sorted(list(detected_skills)),
        "status": detected_status
    }

# ============================
# 5. Costruzione query pesata
# ============================

def build_weighted_query(raw_query, intents):
    base = intents["normalized_query"]
    boosted = [base]

    for skill in intents["skills"]:
        exp = SEMANTIC_EXPANSIONS.get(skill, "")
        w = SEMANTIC_WEIGHTS.get(skill, 1)
        boosted.append((" " + exp) * w)

    return " ".join(boosted).strip()

# ============================
# 6. Motore di ricerca "intelligente"
# ============================

def search_players_scout(query: str, top_k = 10):
    """
    Versione avanzata della search:
    - fa parsing semantico della query
    - costruisce una query pesata
    - calcola similarità coseno contro i profili
    - restituisce risultati + info sugli intenti interpretati
    """
    if not query or not isinstance(query, str):
        raise ValueError("La query deve essere una stringa non vuota.")

    # 1) Parsing semantico
    intents = parse_query_intents(query)

    # 2) Costruzione query pesata
    weighted_query = build_weighted_query(query, intents)

    # 3) TF-IDF della query pesata
    q_vec = vectorizer.transform([weighted_query])
    sim_scores = cosine_similarity(q_vec, tfidf_matrix).ravel()

    # 4) Ordina
    top_idx = np.argsort(sim_scores)[::-1]
    results = metadata_df.iloc[top_idx].copy()
    results["similarity"] = sim_scores[top_idx]

    # FILTRO PER STATUS (Active / Retired)
    if intents["status"] and "Status" in results.columns:
        results = results[results["Status"] == intents["status"]]

    return results.head(top_k), intents, weighted_query





Indice caricato.
TF-IDF matrix shape: (8323, 22019)
Metadata shape: (8323, 7)


In [2]:
# ============================
# 7. Test 
# ============================

query_test = "cerco un buon tiratore da 3 e buon passatore ancora in attività"

results, intents, wq = search_players_scout(query_test)

print("Query:", query_test)
print("\nIntenti interpretati:")
print(" - Skills:", intents.get("skills"))
print(" - Status rilevato:", intents.get("status"))   
print(" - Query normalizzata:", intents.get("normalized_query"))
print("\nQuery pesata:")
print(wq[:200], "...")

cols = ["Player", "DraftYear", "Pick", "similarity", "Status", "text_profile"]
cols = [c for c in cols if c in results.columns]

print("\nTOP risultati:")
display(results[cols])

Query: cerco un buon tiratore da 3 e buon passatore ancora in attività

Intenti interpretati:
 - Skills: ['playmaking', 'shooting_3']
 - Status rilevato: Active
 - Query normalizzata: cerco un buon tiratore da 3 e buon passatore ancora in attività

Query pesata:
cerco un buon tiratore da 3 e buon passatore ancora in attività  assist buon passatore ottimo playmaker assist buon passatore ottimo playmaker assist buon passatore ottimo playmaker  ottimo tiratore d ...

TOP risultati:


,Player,DraftYear,Pick,similarity,Status,text_profile
7133,Chris Paul,2005,4,0.276670,Active,Giocatore: Chris Paul. Proveniente da Wake For...
7376,Stephen Curry,2009,7,0.276443,Active,Giocatore: Stephen Curry. Proveniente da David...
7914,Trae Young,2018,5,0.274384,Active,Giocatore: Trae Young. Proveniente da Oklahoma...
8031,LaMelo Ball,2020,3,0.272490,Active,Giocatore: LaMelo Ball. Proveniente da High Sc...
8094,Josh Giddey,2021,6,0.246398,Active,Giocatore: Josh Giddey. Proveniente da High Sc...
7974,Darius Garland,2019,5,0.246011,Active,Giocatore: Darius Garland. Proveniente da Vand...
7386,Jrue Holiday,2009,17,0.245732,Active,Giocatore: Jrue Holiday. Proveniente da UCLA. ...
7313,Russell Westbrook,2008,4,0.243552,Active,Giocatore: Russell Westbrook. Proveniente da U...
7213,Kyle Lowry,2006,24,0.242160,Active,Giocatore: Kyle Lowry. Proveniente da Villanov...
7971,Ja Morant,2019,2,0.232572,Active,Giocatore: Ja Morant. Proveniente da Murray St...
